In [ ]:
# STEP 1: Setup and imports
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# CONFIGURATION - Change these values for each user
USER_ID = 0  # ⚠️ CHANGE THIS FOR EACH USER
TCX_FILE = f'data/{USER_ID}-d.tcx'
CHART_IMAGE = f'charts_cropped/user_{USER_ID}.png'
OUTPUT_CSV = f'output/processed/user_{USER_ID}_station_data.csv'

# Set working directory
os.chdir('/Users/anthonymccrovitz/Desktop/Sphery/Sphere Heart Rate Analysis')

# Add scripts directory to path
sys.path.append('scripts')
from parse_tcx import parse_tcx_to_df

# Load data
try:
    df, session_total_sec, sessions_avg_hr, session_max_hr, calories_burned = parse_tcx_to_df(TCX_FILE)
    session_duration_min = session_total_sec / 60
    print(f"Data loaded successfully: {len(df)} data points over {session_duration_min:.2f} minutes")
    print(f"Average HR: {sessions_avg_hr:.1f} bpm, Maximum HR: {session_max_hr} bpm, Calories: {calories_burned}")
except Exception as e:
    print(f"Error loading data: {e}")
    
# Load background image
try:
    img = mpimg.imread(CHART_IMAGE)
    print(f"Background image loaded successfully from {CHART_IMAGE}")
except Exception as e:
    print(f"Error loading background image: {e}")


In [ ]:
# Load user metadata
import pandas as pd
try:
    metadata_df = pd.read_csv('metadata/user_metadata.csv')
    user_meta = metadata_df[metadata_df['user_id'] == USER_ID]
    if not user_meta.empty:
        user_meta = user_meta.iloc[0]
        age = user_meta['age'] if not pd.isna(user_meta['age']) else None
        gender = user_meta['gender'] if not pd.isna(user_meta['gender']) else None
        height_cm = user_meta['height_cm'] if not pd.isna(user_meta['height_cm']) else None
        weight_kg = user_meta['weight_kg'] if not pd.isna(user_meta['weight_kg']) else None
        champ_number = user_meta['champ_number'] if not pd.isna(user_meta['champ_number']) else None
        print(f"Loaded metadata for user {USER_ID}: age={age}, gender={gender}, height={height_cm}cm, weight={weight_kg}kg, champ={champ_number}")
    else:
        print(f"No metadata found for user {USER_ID}")
        age = gender = height_cm = weight_kg = champ_number = None
except Exception as e:
    print(f"Error loading metadata: {e}")
    age = gender = height_cm = weight_kg = champ_number = None


In [ ]:
# Create output directories for plots
import os
plots_dir = f'output/plots/user_{USER_ID}'
os.makedirs(plots_dir, exist_ok=True)
print(f"Created plots directory: {plots_dir}")


In [ ]:
# STEP 2: Visualize heart rate data
plt.figure(figsize=(14,5))
plt.plot(df['elapsed_min'], df['heart_rate'], linewidth=2)
plt.xlabel('Elapsed Minutes', fontsize=12)
plt.ylabel('Heart Rate (BPM)', fontsize=12)
plt.title(f'Heart Rate Over Time: User {USER_ID}', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
# Save the plot
plt.savefig(f'{plots_dir}/heart_rate_over_time.png', dpi=300, bbox_inches='tight')
plt.show()

# Show max heart rate point
max_hr_row = df[df['heart_rate'] == df['heart_rate'].max()]
print("Maximum Heart Rate Details:")
print(max_hr_row)


In [ ]:
# STEP 3: DATA QUALITY ASSESSMENT

print("=" * 80)
print(f"USER {USER_ID} DATA QUALITY ASSESSMENT")
print("=" * 80)

print("\n🚨 LOW QUALITY DATA DETECTED 🚨")
print("\nReason: Heart rate data quality is insufficient for reliable analysis.")
print("The HR data shows irregular recording patterns, likely due to:")
print("- Weak or lost connection with the Garmin HR sensor")
print("- Missing or distorted peaks")
print("- Activity zones that are not clearly distinguishable")

print(f"\nData Summary:")
print(f"- Total data points: {len(df)}")
print(f"- Session duration: {session_duration_min:.2f} minutes")
print(f"- Average HR: {sessions_avg_hr:.1f} bpm")
print(f"- Maximum HR: {session_max_hr} bpm")
print(f"- Data points per minute: {len(df) / session_duration_min:.1f}")

print(f"\n⚠️  ANALYSIS:")
print(f"With only {len(df)} data points over {session_duration_min:.2f} minutes,")
print(f"this represents {len(df) / session_duration_min:.1f} data points per minute.")
print(f"Normal HR recording should have ~12-60 data points per minute.")
print(f"This sparse data indicates significant sensor connection issues.")

print("\n📊 APPROACH:")
print("Creating CSV with same format as other users but marking data quality issues.")


In [ ]:
# STEP 4: STATION CUTOFFS FOR LOW-QUALITY DATA (SAME CSV FORMAT)

print("\n" + "=" * 80)
print("STATION PROCESSING - LOW QUALITY DATA")
print("=" * 80)

print("\n📋 CREATING SAME CSV FORMAT AS OTHER USERS")
print("\nApproach for low-quality data:")
print("✅ Create 3 station rows (same as other users)")
print("✅ Use estimated time boundaries")
print("✅ Mark data quality issues in notes/data_quality fields")
print("✅ Keep all survey fields ready for data entry")

# Define estimated station cutoffs 
# Using typical session structure even though data is sparse
cutoffs = [
    (0, 14),    # Station 1 - estimated
    (15, 28),   # Station 2 - estimated  
    (29, 42)    # Station 3 - estimated
]

print(f"\n📅 ESTIMATED STATION BOUNDARIES:")
for i, (start, end) in enumerate(cutoffs, 1):
    print(f"Station {i}: {start:.1f} - {end:.1f} minutes")

# Notes and data quality - UPDATE THESE FOR EACH USER
notes = f"""User {USER_ID}'s heart rate data was explored but not aligned or split into stations. The session shows irregular HR recording, likely due to a weak or lost connection with the Garmin HR sensor. Peaks are missing or distorted, and activity zones are not clearly distinguishable."""

data_quality = """Incomplete heart rate data with long flat sections and dropouts. Unable to detect 6 gameplay peaks or station transitions. Data is unreliable for station-level or gameplay-level analysis. Marked as a low-quality session due to poor HR sensor connection."""

print(f"\n📊 RESULT:")
print("CSV will have 3 rows (same format as other users)")
print("Survey fields ready for data collection")
print("Data quality issues properly documented")


In [ ]:
# STEP 5: CREATE CSV OUTPUT (SAME FORMAT AS HIGH-QUALITY USERS)

print("\n" + "=" * 80)
print("CSV OUTPUT GENERATION")
print("=" * 80)

# Get session timing info from TCX data
session_start_time = df['timestamp'].iloc[0] if len(df) > 0 else None
session_end_time = df['timestamp'].iloc[-1] if len(df) > 0 else None

print(f"📅 SESSION TIMING:")
print(f"Session start: {session_start_time}")
print(f"Session end: {session_end_time}")
print(f"Session duration: {session_duration_min:.2f} minutes")

# Create CSV with same format as high-quality users (3 station rows)
csv_rows = []

for i, (start_min, end_min) in enumerate(cutoffs, 1):
    # Calculate basic stats for this time period (even with sparse data)
    station_data = df[(df['elapsed_min'] >= start_min) & (df['elapsed_min'] <= end_min)]
    
    if len(station_data) > 0:
        station_avg_hr = round(station_data['heart_rate'].mean(), 1)
        station_max_hr = int(station_data['heart_rate'].max())
        # Calculate station start/end timestamps
        station_start_time = station_data['timestamp'].iloc[0]
        station_end_time = station_data['timestamp'].iloc[-1]
    else:
        # No data points in this time range - estimate timestamps
        station_avg_hr = None
        station_max_hr = None
        # Estimate timestamps based on session start + elapsed minutes
        if session_start_time:
            import pandas as pd
            station_start_time = session_start_time + pd.Timedelta(minutes=start_min)
            station_end_time = session_start_time + pd.Timedelta(minutes=end_min)
        else:
            station_start_time = None
            station_end_time = None
    
    # Match high-quality users' column structure EXACTLY (same order and column names)
    row = {
        'user_id': USER_ID,
        'gender': gender if gender and str(gender).strip() else ' ',  # Use space like User 2
        'circuit_type': 'NA',  # Use 'NA' like User 2, you can update this
        'age': age if age is not None else '',  # Empty string if None
        'height_cm': height_cm if height_cm is not None else '',
        'weight_kg': weight_kg if weight_kg is not None else '',
        'champ_number': champ_number,
        'calories_burned': calories_burned,
        'station_number': i,
        'station_name': 'NA',  # Use 'NA' like User 2, you can update this
        'session_start_time': session_start_time,
        'session_end_time': session_end_time,
        'session_duration_min': session_duration_min,
        'session_avg_hr': sessions_avg_hr,
        'session_max_hr': session_max_hr,
        'station_start_time': station_start_time,
        'station_end_time': station_end_time,
        'station_duration_min': end_min - start_min,
        'station_avg_hr': station_avg_hr,
        'station_max_hr': station_max_hr,
        # Survey fields - use 'NA' like User 2 (ready for your data entry)
        'motivation': 'NA',
        'enjoyment': 'NA',
        'team_experience': 'NA',
        'subjective_physical_exertion': 'NA',
        'subjective_cognitive_exertion': 'NA',
        'overall_experience': 'NA',
        'overall_motivation': 'NA',
        'feedback': 'NA',
        'sports_exp': 'NA',
        'gaming_exp': 'NA',
        'data_quality': data_quality,
        'notes': notes
    }
    csv_rows.append(row)

# Convert to DataFrame and save
df_output = pd.DataFrame(csv_rows)
df_output.to_csv(OUTPUT_CSV, index=False)

print(f"✅ CSV file saved to: {OUTPUT_CSV}")
print(f"📊 Record count: {len(df_output)} (3 stations)")
print("\n📋 CSV Structure (matches high-quality users):")
print(f"- User ID: {USER_ID}")
print(f"- Stations: 3 rows")
print(f"- Session timestamps: ✅ Collected from TCX")
print(f"- Station timestamps: ✅ Collected/estimated")
print(f"- Survey fields: Ready for data entry")
print(f"- Data quality: Documented")

print("\n📊 RECORD PREVIEW:")
print(df_output[['user_id', 'station_number', 'session_start_time', 'station_start_time', 'station_duration_min', 'station_avg_hr']].to_string(index=False))


In [ ]:
# STEP 6: FINAL SUMMARY

print("\n" + "=" * 80)
print(f"ANALYSIS COMPLETE - USER {USER_ID}")
print("=" * 80)

print(f"\n✅ Heart rate data processed for User {USER_ID}")
print(f"✅ CSV created with same format as other users")
print(f"✅ Output saved to: {OUTPUT_CSV}")
print("⚠️  Note: Data quality issues documented in CSV")

print(f"\n📊 DATA SUMMARY:")
print(f"User ID: {USER_ID}")
print(f"Data Points: {len(df)}")
print(f"Session Duration: {session_duration_min:.2f} minutes")
print(f"Data Density: {len(df) / session_duration_min:.1f} points/minute")
print(f"Average HR: {sessions_avg_hr:.1f} bpm")
print(f"Maximum HR: {session_max_hr} bpm")

print(f"\n💾 OUTPUT:")
print(f"✅ CSV file: {OUTPUT_CSV}")
print(f"✅ Format: 3 station rows (same as other users)")
print(f"✅ Survey fields: Ready for data entry")
print(f"✅ Data quality: Documented for research")
